In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
import sqlite3
# con = sqlite3.connect('data/raw_data/database.sqlite')

with sqlite3.connect('data/raw_data/database.sqlite') as con:
    matches = pd.read_sql_query("SELECT * from Match where country_id = 21518", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
#select relevant fields

matches = matches[matches.league_id.isin(leagues.id)]

#matches = matches[['id', 'country_id' ,'league_id', 'season', 'stage', 'date','match_api_id', 'home_team_goal', 'away_team_goal','B365H', 'B365D' ,'B365A']]

matches.dropna(inplace=True)

#matches.head()
#matches.count()
players = pd.read_sql_query("SELECT * from Match WHERE home_player_X1 is not null", con)
players.count()

id            24158
country_id    24158
league_id     24158
season        24158
stage         24158
              ...  
GBD           13647
GBA           13647
BSH           13647
BSD           13647
BSA           13647
Length: 115, dtype: int64

In [13]:
matches

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,match_result
1520,23038,21518,21518,2012/2013,1,2012-08-19 00:00:00,1259321,8315,8603,3,...,1.87,3.70,4.33,1.85,3.50,4.10,1.83,3.5,4.20,lose
1521,23039,21518,21518,2012/2013,1,2012-08-19 00:00:00,1259332,8633,10267,1,...,1.25,6.50,12.00,1.26,5.75,9.75,1.25,5.5,11.00,draw
1522,23040,21518,21518,2012/2013,1,2012-08-19 00:00:00,1259353,8634,8560,5,...,1.10,12.00,31.00,1.08,10.00,23.00,1.10,9.0,21.00,win
1523,23041,21518,21518,2012/2013,1,2012-08-18 00:00:00,1259354,8302,8305,2,...,1.67,3.80,6.00,1.60,3.75,5.50,1.70,3.6,5.00,win
1524,23042,21518,21518,2012/2013,1,2012-08-19 00:00:00,1259365,8581,9906,1,...,3.40,3.40,2.25,3.10,3.40,2.30,3.00,3.3,2.30,draw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,23413,21518,21518,2012/2013,9,2012-10-28 00:00:00,1260116,8581,7878,3,...,2.05,3.50,3.90,2.00,3.20,3.90,2.00,3.4,3.50,win
1896,23414,21518,21518,2012/2013,9,2012-10-28 00:00:00,1260118,8394,8302,2,...,3.20,3.40,2.38,3.30,3.20,2.35,2.88,3.3,2.38,win
1897,23415,21518,21518,2012/2013,9,2012-10-28 00:00:00,1260121,8661,8633,0,...,9.00,5.25,1.36,8.75,4.75,1.40,8.50,4.5,1.36,lose
1898,23416,21518,21518,2012/2013,9,2012-10-27 00:00:00,1260123,9910,9783,1,...,1.95,3.60,4.20,1.90,3.40,4.00,1.91,3.5,3.75,draw


# Baseline framework
using SVM machine learning.

In [3]:
from sklearn import svm
def who_win(x):
    if x['home_team_goal'] > x['away_team_goal']:
        return 'win'
    if x['home_team_goal'] < x['away_team_goal']:
        return 'lose'
    if x['home_team_goal'] == x['away_team_goal']:
        return 'draw'

matches['match_result']= matches.apply(lambda x: who_win(x), axis=1)
matches_train = matches.sample(frac=0.7)
matches_test  = matches[~matches.index.isin(matches_train.index)]
clf = svm.SVC()
matches_train.shape
#X = matches['']
#clf.fit(X, Y)

(255, 116)

In [4]:
matches_train.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA',
       'match_result'],
      dtype='object', length=116)

In [5]:
matches_test.shape


(109, 116)

In [6]:
X_train = matches_train[['home_team_api_id','away_team_api_id']]
Y_train = matches_train['match_result']
clf.fit(X_train, Y_train)

SVC()

In [7]:
# clf.score(X_test, y_test)

In [8]:
X_test = matches_test[['home_team_api_id','away_team_api_id']]
matches_test['predicted'] = clf.predict(X_test)

/tmp/ipykernel_29763/202400012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_test['predicted'] = clf.predict(X_test)


In [9]:
matches_test['predict_good'] = matches_test.apply(lambda x: x['match_result']== x['predicted'], axis=1)

/tmp/ipykernel_29763/144563421.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_test['predict_good'] = matches_test.apply(lambda x: x['match_result']== x['predicted'], axis=1)


In [10]:
rate = len(matches_test[matches_test['predict_good']])/len(matches_test)
print (rate)

0.46788990825688076


# Model v2: trying to see correlation with date

## Correlation with month

In [11]:
clf = svm.SVC()
X_train = matches_train[['date','home_team_api_id','away_team_api_id']]
X_train['date'] = X_train.apply(lambda x: x['date'].split('-')[1], axis=1)
Y_train = matches_train['match_result']
clf.fit(X_train, Y_train)

/tmp/ipykernel_29763/1400106515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['date'] = X_train.apply(lambda x: x['date'].split('-')[1], axis=1)


SVC()

In [12]:
X_test = matches_test[['date','home_team_api_id','away_team_api_id']]
X_test['date'] = X_test.apply(lambda x: x['date'].split('-')[1], axis=1)
matches_test['predicted'] = clf.predict(X_test)
matches_test['predict_good'] = matches_test.apply(lambda x: x['match_result']== x['predicted'], axis=1)
rate = len(matches_test[matches_test['predict_good']])/len(matches_test)
print (rate)

0.46788990825688076


/tmp/ipykernel_29763/2742017612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['date'] = X_test.apply(lambda x: x['date'].split('-')[1], axis=1)
/tmp/ipykernel_29763/2742017612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_test['predicted'] = clf.predict(X_test)
/tmp/ipykernel_29763/2742017612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation